## Finetune Falcon-7b on a Google colab

Welcome to this Google Colab notebook that shows how to fine-tune the recent Falcon-7b model on a single Google colab and turn it into a chatbot

We will leverage PEFT library from Hugging Face ecosystem, as well as QLoRA for more memory efficient finetuning

In [1]:
!pip install -q -U trl transformers accelerate git+https://github.com/huggingface/peft.git
!pip install -q datasets bitsandbytes einops wandb

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 245.2/245.2 kB 6.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 297.6/297.6 kB 12.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 14.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 102.0/102.0 kB 13.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 12.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 15.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 15.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 388.9/388.9 kB 16.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.8/119.8 MB 8.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.6/44.6 kB 6.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

## Loading the model

In this section we will load the Falcon 7b Sharded, quantize it in 4bit and attach LoRA adapters on it. Let's get started!

Below we will load the configuration file in order to create the LoRA model. According to QLoRA paper, it is important to consider all linear layers in the transformer block for maximum performance. Therefore we will add `dense`, `dense_h_to_4_h` and `dense_4h_to_h` layers in the target modules in addition to the mixed query key value layer.

In [2]:
from peft import LoraConfig

lora_alpha = 16
lora_dropout = 0.1
lora_r = 64

peft_config = LoraConfig(
    lora_alpha=lora_alpha,
    lora_dropout=lora_dropout,
    r=lora_r,
    bias="none",
    task_type="CAUSAL_LM",
    target_modules=[
        "query_key_value",
        "dense",
        "dense_h_to_4h",
        "dense_4h_to_h",
    ]
)

In [3]:

def print_trainable_parameters(model):
  """
  Prints the number of trainable parameters in the model.
  """
  trainable_params = 0
  all_param = 0
  for _, param in model.named_parameters():
    all_param += param.numel()
    if param.requires_grad:
      trainable_params += param.numel()
  print(
      f"trainable params: {trainable_params} || all params: {all_param} || trainables%: {100 * trainable_params / all_param}"
  )

In [4]:
from peft import (
    LoraConfig,
    PeftConfig,
    PeftModel,
    get_peft_model,
    prepare_model_for_kbit_training
)



In [14]:
import transformers

In [15]:
from pprint import pprint

In [16]:

import json
import os
import torch
import torch.nn as nn

In [13]:
from peft import (
    LoraConfig,
    PeftConfig,
    PeftModel,
    get_peft_model,
    prepare_model_for_kbit_training
)
from transformers import (
    AutoConfig,
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig
)

In [5]:

prompt = """
: midjourney prompt for a girl sit on the mountain
:
""".strip()

In [20]:
MODEL_NAME = "vilsonrodrigues/falcon-7b-instruct-sharded"

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

model = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME,
    device_map="auto",
    trust_remote_code=True,
    quantization_config=bnb_config
)

tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
tokenizer.pad_token = tokenizer.eos_token

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/1.05k [00:00<?, ?B/s]

configuration_falcon.py:   0%|          | 0.00/6.70k [00:00<?, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/vilsonrodrigues/falcon-7b-instruct-sharded:
- configuration_falcon.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


modeling_falcon.py:   0%|          | 0.00/56.9k [00:00<?, ?B/s]

...

Loading checkpoint shards:   0%|          | 0/15 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/117 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/287 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.73M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/281 [00:00<?, ?B/s]

In [6]:
from datasets import load_dataset

In [7]:
data = load_dataset("csv", data_files="dataset.csv")

Generating train split: 0 examples [00:00, ? examples/s]

In [8]:
data

DatasetDict({
    train: Dataset({
        features: ['User', 'Trump'],
        num_rows: 120
    })
})

In [9]:
data["train"][0]

{'User': '"Mr. Trump, what was your reaction to the overwhelming support you received in Michigan "',
 'Trump': '"Thank you, Michigan. What a victory we had in Michigan. What a victory was that. One of the greats. Was that the greatest evening " '}

In [10]:
def generate_prompt(data_point):
  return f"""
: {data_point["User"]}
: {data_point["Trump"]}
""".strip()

def generate_and_tokenize_prompt(data_point):
  full_prompt = generate_prompt(data_point)
  tokenized_full_prompt = tokenizer(full_prompt, padding=True, truncation=True)
  return tokenized_full_prompt

In [21]:
data = data["train"].shuffle().map(generate_and_tokenize_prompt)

Map:   0%|          | 0/120 [00:00<?, ? examples/s]

In [22]:

data

Dataset({
    features: ['User', 'Trump', 'input_ids', 'attention_mask'],
    num_rows: 120
})

In [23]:

model.gradient_checkpointing_enable()
model = prepare_model_for_kbit_training(model)

You are using an old version of the checkpointing format that is deprecated (We will also silently ignore `gradient_checkpointing_kwargs` in case you passed it).Please update to the new format on your modeling file. To use the new format, you need to completely remove the definition of the method `_set_gradient_checkpointing` in your model.
You are using an old version of the checkpointing format that is deprecated (We will also silently ignore `gradient_checkpointing_kwargs` in case you passed it).Please update to the new format on your modeling file. To use the new format, you need to completely remove the definition of the method `_set_gradient_checkpointing` in your model.


In [24]:

config = LoraConfig(
    r=16,
    lora_alpha=32,
    target_modules=["query_key_value"],
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM"
)

model = get_peft_model(model, config)
print_trainable_parameters(model)

trainable params: 4718592 || all params: 3613463424 || trainables%: 0.13058363808693696


In [25]:

prompt = """
: midjourney prompt for a girl sit on the mountain
:
""".strip()

In [26]:

generation_config = model.generation_config
generation_config.max_new_tokens = 200
generation_config.temperature = 0.7
generation_config.top_p = 0.7
generation_config.num_return_sequences = 1
generation_config.pad_token_id = tokenizer.eos_token_id
generation_config.eos_token_id = tokenizer.eos_token_id

In [27]:

%%time
device = "cuda:0"

encoding = tokenizer(prompt, return_tensors="pt").to(device)
with torch.no_grad():
  outputs = model.generate(
      input_ids = encoding.input_ids,
      attention_mask = encoding.attention_mask,
      generation_config = generation_config
  )

print(tokenizer.decode(outputs[0], skip_special_tokens=True))


/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:492: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.7` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:497: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.7` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(


: midjourney prompt for a girl sit on the mountain
: midjourney prompt for a girl sit on the mountain
: midjourney prompt for a girl sit on the mountain
: midjourney prompt for a girl sit on the mountain
: midjourney prompt for a girl sit on the mountain
: midjourney prompt for a girl sit on the mountain
: midjourney prompt for a girl sit on the mountain
: midjourney prompt for a girl sit on the mountain
: midjourney prompt for a girl sit on the mountain
: midjourney prompt for a girl sit on the mountain
: midjourney prompt for a girl sit on the mountain
: midjourney prompt for a girl sit on the mountain
: midjourney prompt for a girl sit on the mountain
: midjourney prompt for a girl sit on the mountain
: midjourney prompt for a girl sit on the mountain
: midjourney prompt for a girl sit on the mountain
: midjourney prompt for
CPU times: user 25.6 s, sys: 473 ms, total: 26.1 s
Wall time: 30.7 s


In [28]:
training_args = transformers.TrainingArguments(
      per_device_train_batch_size=1,
      gradient_accumulation_steps=4,
      num_train_epochs=1,
      learning_rate=2e-4,
      fp16=True,
      save_total_limit=3,
      logging_steps=1,
      output_dir="experiments",
      optim="paged_adamw_8bit",
      lr_scheduler_type="cosine",
      warmup_ratio=0.05,
)

trainer = transformers.Trainer(
    model=model,
    train_dataset=data,
    args=training_args,
    data_collator=transformers.DataCollatorForLanguageModeling(tokenizer, mlm=False)
)
model.config.use_cache = False
trainer.train()

<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:460: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(


Step,Training Loss
1,2.705600
2,1.781900
3,2.350200
4,2.301300
5,2.516400
6,2.400000
7,2.290100
8,2.589000
9,2.489200
10,2.238900


TrainOutput(global_step=30, training_loss=2.223699990908305, metrics={'train_runtime': 288.1111, 'train_samples_per_second': 0.417, 'train_steps_per_second': 0.104, 'total_flos': 555489638945280.0, 'train_loss': 2.223699990908305, 'epoch': 1.0})

In [29]:
model.save_pretrained("Trump-trained-model")

In [30]:
from huggingface_hub import notebook_login

In [31]:

notebook_login()

In [32]:
model.save_pretrained("Trump-trained-model")

In [33]:

PEFT_MODEL = "hamzahey/Falcon_7b_Instruct_sharded"

model.push_to_hub(
    PEFT_MODEL, use_auth_token=True
)

/usr/local/lib/python3.10/dist-packages/transformers/utils/hub.py:834: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(


adapter_model.safetensors:   0%|          | 0.00/18.9M [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/hamzahey/Falcon_7b_Instruct_sharded/commit/a0bc2fc335a9e2e87fda64a602b68662609255bb', commit_message='Upload model', commit_description='', oid='a0bc2fc335a9e2e87fda64a602b68662609255bb', pr_url=None, pr_revision=None, pr_num=None)

In [34]:
from huggingface_hub import notebook_login
notebook_login()

In [35]:

config = PeftConfig.from_pretrained(PEFT_MODEL)
model = AutoModelForCausalLM.from_pretrained(
    config.base_model_name_or_path,
    return_dict=True,
    quantization_config=bnb_config,
    device_map="auto",
    trust_remote_code=True
)

tokenizer=AutoTokenizer.from_pretrained(config.base_model_name_or_path)
tokenizer.pad_token = tokenizer.eos_token

model = PeftModel.from_pretrained(model, PEFT_MODEL)


adapter_config.json:   0%|          | 0.00/663 [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/15 [00:00<?, ?it/s]

adapter_model.safetensors:   0%|          | 0.00/18.9M [00:00<?, ?B/s]

In [36]:

generation_config = model.generation_config
generation_config.max_new_tokens = 20
generation_config.temperature = 0.7
generation_config.top_p = 0.7
generation_config.num_return_sequences = 1
generation_config.pad_token_id = tokenizer.eos_token_id
generation_config.eos_token_id = tokenizer.eos_token_id
generation_config.num_return_sequences = 1

In [38]:
%%time
device = "cuda:0"

prompt = """
: where are you speaking from
:
""".strip()

encoding = tokenizer(prompt, return_tensors="pt").to(device)
with torch.no_grad():
  outputs = model.generate(
      input_ids = encoding.input_ids,
      attention_mask = encoding.attention_mask,
      generation_config = generation_config
  )

generated_response = tokenizer.decode(outputs[0], skip_special_tokens=True)
print(generated_response)



: where are you speaking from
: I'm in the city of New York
: 
CPU times: user 1.94 s, sys: 0 ns, total: 1.94 s
Wall time: 2.11 s


In [41]:
%%time
device = "cuda:0"

prompt = """
: Mr. Trump, how do you intend to address concerns about education and patriotic values in schools?
:
""".strip()

encoding = tokenizer(prompt, return_tensors="pt").to(device)
with torch.no_grad():
  outputs = model.generate(
      input_ids = encoding.input_ids,
      attention_mask = encoding.attention_mask,
      generation_config = generation_config
  )

generated_response = tokenizer.decode(outputs[0], skip_special_tokens=True)
print(generated_response)

: Mr. Trump, how do you intend to address concerns about education and patriotic values in schools?
: I believe that we need to focus on making sure that our schools are properly funded and that our teachers
CPU times: user 2.52 s, sys: 15 µs, total: 2.52 s
Wall time: 3.28 s


In [42]:
%%time
device = "cuda:0"

prompt = """
: Mr. Trump, what do you think about politics?
:
""".strip()

encoding = tokenizer(prompt, return_tensors="pt").to(device)
with torch.no_grad():
  outputs = model.generate(
      input_ids = encoding.input_ids,
      attention_mask = encoding.attention_mask,
      generation_config = generation_config
  )

generated_response = tokenizer.decode(outputs[0], skip_special_tokens=True)
print(generated_response)

: Mr. Trump, what do you think about politics?
: I think it's a great thing to be involved in. I think it's a great
CPU times: user 2.68 s, sys: 2.79 ms, total: 2.69 s
Wall time: 2.9 s


In [46]:
%%time
device = "cuda:0"

prompt = """
: Mr. Trump, what's your response to claims that Minnesota is in play for this election?
:
""".strip()

encoding = tokenizer(prompt, return_tensors="pt").to(device)
with torch.no_grad():
  outputs = model.generate(
      input_ids = encoding.input_ids,
      attention_mask = encoding.attention_mask,
      generation_config = generation_config
  )

generated_response = tokenizer.decode(outputs[0], skip_special_tokens=True)
print(generated_response)

: Mr. Trump, what's your response to claims that Minnesota is in play for this election?
: I'm not going to comment on that. I'm going to let the people of Minnesota
CPU times: user 2.61 s, sys: 0 ns, total: 2.61 s
Wall time: 4.16 s


During training, the model should converge nicely as follows:

![image](https://huggingface.co/datasets/trl-internal-testing/example-images/resolve/main/images/loss-falcon-7b.png)

The `Trainer` also takes care of properly saving only the adapters during training instead of saving the entire model.